# Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Explore Data

In [4]:
basics.info()
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
akas.info()
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.info()
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1925
1,tt0000002,5.8,261
2,tt0000003,6.5,1741
3,tt0000004,5.6,176
4,tt0000005,6.2,2554


# Preprocessing

## Replace "\N" with np.nan

In [7]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

In [8]:
basics.replace({'\\N':np.nan}, inplace=True)

In [9]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1265632
endYear           9346581
runtimeMinutes    6756650
genres             431735
dtype: int64

In [10]:
akas.replace({'\\N':np.nan}, inplace=True)

In [11]:
ratings.replace({'\\N':np.nan}, inplace=True)

## Exclude any movie with missing values for genre or runtime

In [12]:
basics = basics[(basics['runtimeMinutes'].notna())]

In [13]:
basics = basics[(basics['genres'].notna())]

## Include only full-length movies (titleType = "movie")

In [14]:
basics = basics[(basics['titleType']=='Movie')]

## Include only movies that were released 2000 - 2021 (include 2000 and 2021)

In [15]:
basics = basics[(basics['startYear']>=2000) & (basics['startYear']<=2022)]

## Include only fictional movies (not from documentary genre)

In [16]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

## Include only movies that were released in the United States

In [17]:
akas = akas[(akas['region']=='US')]

In [18]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

Series([], Name: tconst, dtype: bool)

In [19]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [20]:
ratings_keepers =ratings['tconst'].isin(akas['titleId'])
ratings_keepers

0           True
1           True
2          False
3          False
4           True
           ...  
1257083    False
1257084     True
1257085    False
1257086    False
1257087    False
Name: tconst, Length: 1257088, dtype: bool

In [21]:
ratings = ratings[ratings_keepers]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1925
1,tt0000002,5.8,261
4,tt0000005,6.2,2554
5,tt0000006,5.1,175
6,tt0000007,5.4,797
...,...,...,...
1257062,tt9916204,8.2,250
1257068,tt9916348,8.5,17
1257069,tt9916362,6.4,5050
1257073,tt9916428,3.8,14


# Final Summary after Filtering

In [22]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          0 non-null      object
 1   titleType       0 non-null      object
 2   primaryTitle    0 non-null      object
 3   originalTitle   0 non-null      object
 4   isAdult         0 non-null      object
 5   startYear       0 non-null      object
 6   endYear         0 non-null      object
 7   runtimeMinutes  0 non-null      object
 8   genres          0 non-null      object
dtypes: object(9)
memory usage: 0.0+ bytes


In [23]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392212 entries, 5 to 34183296
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1392212 non-null  object
 1   ordering         1392212 non-null  int64 
 2   title            1392212 non-null  object
 3   region           1392212 non-null  object
 4   language         3738 non-null     object
 5   types            967379 non-null   object
 6   attributes       45511 non-null    object
 7   isOriginalTitle  1390867 non-null  object
dtypes: int64(1), object(7)
memory usage: 95.6+ MB


In [24]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478753 entries, 0 to 1257084
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         478753 non-null  object 
 1   averageRating  478753 non-null  float64
 2   numVotes       478753 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB
